In [1]:
import numpy as np
from collections import defaultdict

input_file = "data/input.txt"

BROADCASTER = "broadcaster"
FLIPFLOP = "%"
CONJUNCTION = "&"

def get_module(line):
    if line == BROADCASTER:
        return (BROADCASTER, BROADCASTER)
    module_type, module_name = line[0], line[1:]
    return (module_type, module_name)

def split_line(line):
    [module, outputs] = line.split(" -> ")
    module = get_module(module)
    outputs = outputs.split(", ")
    return module, outputs

def parse_modules(lines):
    modules = defaultdict(lambda: {"module_type": None})
    for l in lines:
        (module_type, module_name), outputs = split_line(l)
        modules[module_name] = { "module_type": module_type, "outputs": outputs }
        if module_type == FLIPFLOP:
            modules[module_name]["state"] = False
        elif module_type == CONJUNCTION:
            modules[module_name]["state"] = {}

    for module in modules:
        if modules[module]["module_type"] == CONJUNCTION:
            for module2 in modules:
                if module in modules[module2]["outputs"]:
                    modules[module]["state"][module2] = False
    return modules

def get_second_deps(modules):
    first_deps = []
    second_deps = []
    for m in modules:
        if "outputs" in modules[m]:
            if "rx" in modules[m]["outputs"]:
                first_deps.append(m)

    for dep in first_deps:
        for m in modules:
            if "outputs" in modules[m]:
                if dep in modules[m]["outputs"]:
                    second_deps.append(m)
    return second_deps

def simulate_button_press(modules, current_push, recurrences):
    high_pulses = 0
    low_pulses = 0
    pulses = [(False, BROADCASTER, None)]

    while len(pulses) > 0:
        high, target_name, from_name = pulses.pop(0)
        if from_name in recurrences.keys() and high:
            recurrences[from_name] = min(current_push, recurrences[from_name])

        target_type = modules[target_name]["module_type"]
        if high:
            high_pulses += 1
        else:
            low_pulses += 1
        if target_type == BROADCASTER:
            for connection in modules[target_name]["outputs"]:
                pulses.append((high, connection, target_name))
        elif target_type == FLIPFLOP:
            if not high:
                modules[target_name]["state"] = not modules[target_name]["state"]
                for connection in modules[target_name]["outputs"]:
                    pulses.append((modules[target_name]["state"], connection, target_name))
        elif target_type == CONJUNCTION:
            modules[target_name]["state"][from_name] = high
            states = modules[target_name]["state"].values()
            if all(states):
                for connection in modules[target_name]["outputs"]:
                    pulses.append((False, connection, target_name))
            else:
                for connection in modules[target_name]["outputs"]:
                    pulses.append((True, connection, target_name))
    return high_pulses, low_pulses

with open(input_file, 'r') as f:
    lines = [l.strip() for l in f.readlines()]
    modules = parse_modules(lines)
    
    high_pulses = 0
    low_pulses = 0
    button_presses = 0

    second_deps = get_second_deps(modules)
    recurrences = {d: float('inf') for d in second_deps}

    for _ in range(1000):
        button_presses += 1
        h, l = simulate_button_press(modules, button_presses, recurrences)
        high_pulses += h
        low_pulses += l

    while not all(x < float('inf') for x in recurrences.values()):
        button_presses += 1
        simulate_button_press(modules, button_presses, recurrences)

    recurrences = list(recurrences.values())
    ans1 = high_pulses * low_pulses
    ans2 = np.lcm.reduce(recurrences)

    print(f"{ans1 = }")
    print(f"{ans2 = }")

ans1 = 711650489
ans2 = 219388737656593
